In [2]:
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv('model1_data.csv')

In [5]:
data.head(2)

,Unnamed: 0,name,smiles,avg_mass,logP,logS,PSA,HBD,HBA,rot_bonds,...,temperature_min,temperature_max,duration_min,duration_max,strength_min,strength_max,pH_min,pH_max,freq_weight_mean,predicted_api_loss_percent
0,0,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,...,25.0,25.0,2.0,2.0,1.00,1.00,0.5,0.5,0.007826,5.0
1,1,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,...,25.0,25.0,6.0,6.0,0.01,0.01,2.0,2.0,0.007826,3.0


In [3]:
data_gnn=data.iloc[:,:13]
data_stress=data.iloc[:,14:]

In [4]:
data_gnn.head()

,Unnamed: 0,name,smiles,avg_mass,logP,logS,PSA,HBD,HBA,rot_bonds,rings,pKa_acid,pKa_base
0,0,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,2.78,11.88
1,1,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,2.78,11.88
2,2,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,2.78,11.88
3,3,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,2.78,11.88
4,4,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,2.78,11.88


In [5]:
data_stress.head()

,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_min,temperature_max,duration_min,duration_max,strength_min,strength_max,pH_min,pH_max,freq_weight_mean,predicted_api_loss_percent
0,acid_hydrolysis,HCl,mild,low,medium,high,25.0,25.0,2.0,2.0,1.00,1.00,0.5,0.5,0.007826,5.0
1,acid_hydrolysis,HCl,mild,low,medium,low,25.0,25.0,6.0,6.0,0.01,0.01,2.0,2.0,0.007826,3.0
2,acid_hydrolysis,HCl,mild,low,medium,medium,25.0,25.0,2.0,24.0,0.10,0.50,0.8,1.2,0.007826,4.0
3,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,60.0,2.0,6.0,0.10,0.10,1.2,1.2,0.016957,10.0
4,acid_hydrolysis,HCl,moderate,high,short,low,60.0,60.0,1.0,1.0,0.01,0.01,2.0,2.0,0.010435,7.0


In [6]:
data_stress['temperature_mean']=(data_stress['temperature_max']+data_stress['temperature_min'])/2
data_stress['duration_mean']=(data_stress['duration_max']+data_stress['duration_min'])/2
data_stress['strength_mean']=(data_stress['strength_max']+data_stress['strength_min'])/2
data_stress['pH_mean']=(data_stress['pH_max']+data_stress['pH_min'])/2

data_stress=data_stress.drop(columns=['temperature_max','temperature_min','duration_max','duration_min','strength_max','strength_min','pH_max','pH_min'])

In [7]:
data_stress.head()

,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,freq_weight_mean,predicted_api_loss_percent,temperature_mean,duration_mean,strength_mean,pH_mean
0,acid_hydrolysis,HCl,mild,low,medium,high,0.007826,5.0,25.0,2.0,1.00,0.5
1,acid_hydrolysis,HCl,mild,low,medium,low,0.007826,3.0,25.0,6.0,0.01,2.0
2,acid_hydrolysis,HCl,mild,low,medium,medium,0.007826,4.0,25.0,13.0,0.30,1.0
3,acid_hydrolysis,HCl,moderate,high,medium,medium,0.016957,10.0,60.0,4.0,0.10,1.2
4,acid_hydrolysis,HCl,moderate,high,short,low,0.010435,7.0,60.0,1.0,0.01,2.0


In [8]:
data_stress.to_csv('ann_data.csv',index=False)

In [24]:
data_stress['temp_bin'].value_counts()

temp_bin
high    2500
na      2500
mid     1900
low     1500
Name: count, dtype: int64

In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv('model_data.csv')

In [3]:
data.head()

,Unnamed: 0.3,Unnamed: 0.1,Unnamed: 0,drugbank_id,name,smiles,avg_mass,logP,logS,PSA,...,strength_max,pH_min,pH_max,freq_weight_mean,light_lux_hr_applied,uv_Wh_m2_applied,humidity_RH_applied,Unnamed: 0.2,predicted_api_loss_percent,predicted_degradant_percent
0,0,0,0,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,...,1.00,0.5,0.5,0.007826,0,0,0,0,4.0,2.0
1,1,1,1,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,...,0.01,2.0,2.0,0.007826,0,0,0,1,2.0,1.0
2,2,2,2,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,...,0.50,0.8,1.2,0.007826,0,0,0,2,5.0,2.5
3,3,3,3,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,...,0.10,1.2,1.2,0.016957,0,0,0,3,8.0,4.0
4,4,4,4,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,...,0.01,2.0,2.0,0.010435,0,0,0,4,6.0,3.0


In [4]:
data=data.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.1', 'Unnamed: 0','Unnamed: 0.2'])

In [7]:
data=data.drop(columns=['drugbank_id'])

In [9]:
data=data.drop(columns=['light_lux_hr_applied', 'uv_Wh_m2_applied', 'humidity_RH_applied'])

In [10]:
data.head()

,name,smiles,avg_mass,logP,logS,PSA,HBD,HBA,rot_bonds,rings,...,temperature_max,duration_min,duration_max,strength_min,strength_max,pH_min,pH_max,freq_weight_mean,predicted_api_loss_percent,predicted_degradant_percent
0,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,...,25.0,2.0,2.0,1.00,1.00,0.5,0.5,0.007826,4.0,2.0
1,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,...,25.0,6.0,6.0,0.01,0.01,2.0,2.0,0.007826,2.0,1.0
2,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,...,25.0,2.0,24.0,0.10,0.50,0.8,1.2,0.007826,5.0,2.5
3,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,...,60.0,2.0,6.0,0.10,0.10,1.2,1.2,0.016957,8.0,4.0
4,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,28.0,37.0,66.0,6.0,...,60.0,1.0,1.0,0.01,0.01,2.0,2.0,0.010435,6.0,3.0


In [11]:
data_gnn=data.iloc[:,:13]
data_stress=data.iloc[:,14:]

In [13]:
data_stress

,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_min,temperature_max,duration_min,duration_max,strength_min,strength_max,pH_min,pH_max,freq_weight_mean,predicted_api_loss_percent,predicted_degradant_percent
0,HCl,mild,low,medium,high,25.0,25.0,2.0,2.0,1.00,1.00,0.5,0.5,0.007826,4.0,2.0
1,HCl,mild,low,medium,low,25.0,25.0,6.0,6.0,0.01,0.01,2.0,2.0,0.007826,2.0,1.0
2,HCl,mild,low,medium,medium,25.0,25.0,2.0,24.0,0.10,0.50,0.8,1.2,0.007826,5.0,2.5
3,HCl,moderate,high,medium,medium,60.0,60.0,2.0,6.0,0.10,0.10,1.2,1.2,0.016957,8.0,4.0
4,HCl,moderate,high,short,low,60.0,60.0,1.0,1.0,0.01,0.01,2.0,2.0,0.010435,6.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,dry_heat,moderate,mid,short,na,50.0,50.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0,1.0
8396,dry_heat,strong,high,long,na,80.0,80.0,72.0,72.0,NaN,NaN,NaN,NaN,0.010435,14.0,5.5
8397,dry_heat,strong,high,medium,na,80.0,110.0,3.0,24.0,NaN,NaN,NaN,NaN,0.012174,16.0,6.0
8398,dry_heat,strong,high,na,na,80.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,5.0


In [14]:
data_stress['temperature_mean']=(data_stress['temperature_max']+data_stress['temperature_min'])/2
data_stress['duration_mean']=(data_stress['duration_max']+data_stress['duration_min'])/2
data_stress['strength_mean']=(data_stress['strength_max']+data_stress['strength_min'])/2
data_stress['pH_mean']=(data_stress['pH_max']+data_stress['pH_min'])/2

data_stress=data_stress.drop(columns=['temperature_max','temperature_min','duration_max','duration_min','strength_max','strength_min','pH_max','pH_min'])

In [15]:
data_stress.head()

,reagent,severity,temp_bin,duration_bin,strength_bin,freq_weight_mean,predicted_api_loss_percent,predicted_degradant_percent,temperature_mean,duration_mean,strength_mean,pH_mean
0,HCl,mild,low,medium,high,0.007826,4.0,2.0,25.0,2.0,1.00,0.5
1,HCl,mild,low,medium,low,0.007826,2.0,1.0,25.0,6.0,0.01,2.0
2,HCl,mild,low,medium,medium,0.007826,5.0,2.5,25.0,13.0,0.30,1.0
3,HCl,moderate,high,medium,medium,0.016957,8.0,4.0,60.0,4.0,0.10,1.2
4,HCl,moderate,high,short,low,0.010435,6.0,3.0,60.0,1.0,0.01,2.0


In [16]:
data_stress.to_csv('ann_data.csv',index=False)

In [17]:
import numpy as np
import pandas as pd


In [18]:
data=pd.read_csv('ann_data.csv')
data1=pd.read_csv('model_data.csv')

In [19]:
data.head(2)

,reagent,severity,temp_bin,duration_bin,strength_bin,freq_weight_mean,predicted_api_loss_percent,predicted_degradant_percent,temperature_mean,duration_mean,strength_mean,pH_mean
0,HCl,mild,low,medium,high,0.007826,4.0,2.0,25.0,2.0,1.00,0.5
1,HCl,mild,low,medium,low,0.007826,2.0,1.0,25.0,6.0,0.01,2.0


In [20]:
data1.head(2)

,Unnamed: 0.3,Unnamed: 0.1,Unnamed: 0,drugbank_id,name,smiles,avg_mass,logP,logS,PSA,...,strength_max,pH_min,pH_max,freq_weight_mean,light_lux_hr_applied,uv_Wh_m2_applied,humidity_RH_applied,Unnamed: 0.2,predicted_api_loss_percent,predicted_degradant_percent
0,0,0,0,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,...,1.00,0.5,0.5,0.007826,0,0,0,0,4.0,2.0
1,1,1,1,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,2180.2853,-14.0,-4.7,901.57,...,0.01,2.0,2.0,0.007826,0,0,0,1,2.0,1.0


In [2]:
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv('model_predictions_complete.csv')

In [4]:
data.head()

,drugbank_id,name,smiles,stress_type,reagent,reagent.1,severity,severity.1,temp_bin,temp_bin.1,...,predicted_degradant_percent,predicted_degradant_percent.1,freq_weight_mean.2,freq_weight_mean.3,predicted_api_loss_percent.2,predicted_api_loss_percent.3,predicted_degradant_percent.2,predicted_degradant_percent.3,api_prediction_error,degradant_prediction_error
0,DB00435,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,HCl,moderate,moderate,mid,mid,...,5.5,5.5,0.015652,0.015652,11.0,11.0,5.5,5.5,3.99,2.58
1,DB00130,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,NaOH,strong,strong,high,high,...,5.0,5.0,0.013043,0.013043,12.0,12.0,5.0,5.0,0.02,0.09
2,DB00209,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,HCl,moderate,moderate,high,high,...,3.5,3.5,0.016957,0.016957,7.0,7.0,3.5,3.5,1.61,1.29
3,DB00194,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,NaOH,moderate,moderate,mid,mid,...,2.0,2.0,0.011739,0.011739,5.0,5.0,2.0,2.0,0.71,0.37
4,DB00443,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,NaOH,strong,strong,high,high,...,6.0,6.0,0.020870,0.020870,15.0,15.0,6.0,6.0,2.80,0.82


In [5]:
data.columns

Index(['drugbank_id', 'name', 'smiles', 'stress_type', 'reagent', 'reagent.1',
       'severity', 'severity.1', 'temp_bin', 'temp_bin.1', 'duration_bin',
       'duration_bin.1', 'strength_bin', 'strength_bin.1', 'temperature_mean',
       'duration_mean', 'strength_mean', 'pH_mean', 'initial_api_percent',
       'initial_degradant_percent', 'true_api_loss_percent',
       'true_degradant_percent', 'model_predicted_api_loss_percent',
       'model_predicted_degradant_percent', 'Unnamed: 0.3', 'Unnamed: 0.1',
       'Unnamed: 0', 'avg_mass', 'logP', 'logS', 'PSA', 'HBD', 'HBA',
       'rot_bonds', 'rings', 'pKa_acid', 'pKa_base', 'phys_charge',
       'temperature_min', 'temperature_max', 'duration_min', 'duration_max',
       'strength_min', 'strength_max', 'pH_min', 'pH_max', 'freq_weight_mean',
       'freq_weight_mean.1', 'light_lux_hr_applied', 'uv_Wh_m2_applied',
       'humidity_RH_applied', 'Unnamed: 0.2', 'predicted_api_loss_percent',
       'predicted_api_loss_percent.1', 'pre

In [6]:
data=data.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.1','Unnamed: 0','temperature_min', 'temperature_max', 'duration_min', 'duration_max','strength_min', 'strength_max', 'pH_min', 'pH_max', 'freq_weight_mean','freq_weight_mean.1', 'light_lux_hr_applied', 'uv_Wh_m2_applied','humidity_RH_applied', 'Unnamed: 0.2'])

In [7]:
data.head()

,drugbank_id,name,smiles,stress_type,reagent,reagent.1,severity,severity.1,temp_bin,temp_bin.1,...,predicted_degradant_percent,predicted_degradant_percent.1,freq_weight_mean.2,freq_weight_mean.3,predicted_api_loss_percent.2,predicted_api_loss_percent.3,predicted_degradant_percent.2,predicted_degradant_percent.3,api_prediction_error,degradant_prediction_error
0,DB00435,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,HCl,moderate,moderate,mid,mid,...,5.5,5.5,0.015652,0.015652,11.0,11.0,5.5,5.5,3.99,2.58
1,DB00130,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,NaOH,strong,strong,high,high,...,5.0,5.0,0.013043,0.013043,12.0,12.0,5.0,5.0,0.02,0.09
2,DB00209,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,HCl,moderate,moderate,high,high,...,3.5,3.5,0.016957,0.016957,7.0,7.0,3.5,3.5,1.61,1.29
3,DB00194,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,NaOH,moderate,moderate,mid,mid,...,2.0,2.0,0.011739,0.011739,5.0,5.0,2.0,2.0,0.71,0.37
4,DB00443,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,NaOH,strong,strong,high,high,...,6.0,6.0,0.020870,0.020870,15.0,15.0,6.0,6.0,2.80,0.82


In [8]:
data=data.drop(columns=['predicted_degradant_percent.1', 'freq_weight_mean.2', 'freq_weight_mean.3', 'predicted_api_loss_percent.2', 'predicted_api_loss_percent.3', 'predicted_degradant_percent.2', 'predicted_degradant_percent.3'])

In [9]:
data.head()

,drugbank_id,name,smiles,stress_type,reagent,reagent.1,severity,severity.1,temp_bin,temp_bin.1,...,rot_bonds,rings,pKa_acid,pKa_base,phys_charge,predicted_api_loss_percent,predicted_api_loss_percent.1,predicted_degradant_percent,api_prediction_error,degradant_prediction_error
0,DB00435,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,HCl,moderate,moderate,mid,mid,...,0.0,0.0,NaN,-2.90,0.0,11.0,11.0,5.5,3.99,2.58
1,DB00130,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,NaOH,strong,strong,high,high,...,4.0,0.0,2.15,9.31,0.0,12.0,12.0,5.0,0.02,0.09
2,DB00209,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,HCl,moderate,moderate,high,high,...,5.0,5.0,11.05,-4.50,1.0,7.0,7.0,3.5,1.61,1.29
3,DB00194,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,NaOH,moderate,moderate,mid,mid,...,2.0,3.0,12.45,3.94,0.0,5.0,5.0,2.0,0.71,0.37
4,DB00443,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,NaOH,strong,strong,high,high,...,2.0,4.0,12.42,-3.30,0.0,15.0,15.0,6.0,2.80,0.82


In [10]:
data.columns

Index(['drugbank_id', 'name', 'smiles', 'stress_type', 'reagent', 'reagent.1',
       'severity', 'severity.1', 'temp_bin', 'temp_bin.1', 'duration_bin',
       'duration_bin.1', 'strength_bin', 'strength_bin.1', 'temperature_mean',
       'duration_mean', 'strength_mean', 'pH_mean', 'initial_api_percent',
       'initial_degradant_percent', 'true_api_loss_percent',
       'true_degradant_percent', 'model_predicted_api_loss_percent',
       'model_predicted_degradant_percent', 'avg_mass', 'logP', 'logS', 'PSA',
       'HBD', 'HBA', 'rot_bonds', 'rings', 'pKa_acid', 'pKa_base',
       'phys_charge', 'predicted_api_loss_percent',
       'predicted_api_loss_percent.1', 'predicted_degradant_percent',
       'api_prediction_error', 'degradant_prediction_error'],
      dtype='object')

In [11]:
data=data.drop(columns=['reagent.1','severity.1','temp_bin.1','duration_bin.1','strength_bin.1','predicted_api_loss_percent.1'])

In [12]:
data.head()

,drugbank_id,name,smiles,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_mean,...,HBA,rot_bonds,rings,pKa_acid,pKa_base,phys_charge,predicted_api_loss_percent,predicted_degradant_percent,api_prediction_error,degradant_prediction_error
0,DB00435,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,moderate,mid,medium,medium,40.0,...,1.0,0.0,0.0,NaN,-2.90,0.0,11.0,5.5,3.99,2.58
1,DB00130,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,strong,high,medium,medium,60.0,...,4.0,4.0,0.0,2.15,9.31,0.0,12.0,5.0,0.02,0.09
2,DB00209,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,...,2.0,5.0,5.0,11.05,-4.50,1.0,7.0,3.5,1.61,1.29
3,DB00194,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,moderate,mid,medium,low,40.0,...,8.0,2.0,3.0,12.45,3.94,0.0,5.0,2.0,0.71,0.37
4,DB00443,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,strong,high,short,medium,60.0,...,5.0,2.0,4.0,12.42,-3.30,0.0,15.0,6.0,2.80,0.82


In [13]:
data.columns

Index(['drugbank_id', 'name', 'smiles', 'stress_type', 'reagent', 'severity',
       'temp_bin', 'duration_bin', 'strength_bin', 'temperature_mean',
       'duration_mean', 'strength_mean', 'pH_mean', 'initial_api_percent',
       'initial_degradant_percent', 'true_api_loss_percent',
       'true_degradant_percent', 'model_predicted_api_loss_percent',
       'model_predicted_degradant_percent', 'avg_mass', 'logP', 'logS', 'PSA',
       'HBD', 'HBA', 'rot_bonds', 'rings', 'pKa_acid', 'pKa_base',
       'phys_charge', 'predicted_api_loss_percent',
       'predicted_degradant_percent', 'api_prediction_error',
       'degradant_prediction_error'],
      dtype='object')

In [ ]:
data['true_api_loss_percent'].corr(data['predicted_api_loss_percent'])

np.float64(1.0)

In [15]:
data['true_degradant_percent'].corr(data['predicted_degradant_percent'])

np.float64(1.0)

In [16]:
data=data.drop(columns=['predicted_api_loss_percent','predicted_degradant_percent', 'api_prediction_error','degradant_prediction_error'])

In [17]:
data.head()

,drugbank_id,name,smiles,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_mean,...,logP,logS,PSA,HBD,HBA,rot_bonds,rings,pKa_acid,pKa_base,phys_charge
0,DB00435,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,moderate,mid,medium,medium,40.0,...,-0.35,NaN,34.14,0.0,1.0,0.0,0.0,NaN,-2.90,0.0
1,DB00130,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,strong,high,medium,medium,60.0,...,-4.00,-0.17,106.41,3.0,4.0,4.0,0.0,2.15,9.31,0.0
2,DB00209,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,...,-0.50,-6.80,46.53,1.0,2.0,5.0,5.0,11.05,-4.50,1.0
3,DB00194,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,moderate,mid,medium,low,40.0,...,-2.10,-1.30,139.54,4.0,8.0,2.0,3.0,12.45,3.94,0.0
4,DB00443,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,strong,high,short,medium,60.0,...,1.68,-3.90,94.83,3.0,5.0,2.0,4.0,12.42,-3.30,0.0


In [18]:
data=data.drop(columns=['phys_charge','drugbank_id'])

In [19]:
data.head()

,name,smiles,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_mean,duration_mean,...,avg_mass,logP,logS,PSA,HBD,HBA,rot_bonds,rings,pKa_acid,pKa_base
0,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,moderate,mid,medium,medium,40.0,6.0,...,30.0061,-0.35,NaN,34.14,0.0,1.0,0.0,0.0,NaN,-2.90
1,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,strong,high,medium,medium,60.0,2.0,...,146.1445,-4.00,-0.17,106.41,3.0,4.0,4.0,0.0,2.15,9.31
2,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,4.0,...,392.5180,-0.50,-6.80,46.53,1.0,2.0,5.0,5.0,11.05,-4.50
3,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,moderate,mid,medium,low,40.0,4.5,...,267.2413,-2.10,-1.30,139.54,4.0,8.0,2.0,3.0,12.45,3.94
4,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,strong,high,short,medium,60.0,1.0,...,392.4611,1.68,-3.90,94.83,3.0,5.0,2.0,4.0,12.42,-3.30


In [20]:
data.to_csv('final_model_data.csv',index=False)

In [21]:
import numpy as np
import pandas as pd

In [22]:
dat_real=pd.read_csv('final_model_data.csv')

In [23]:
dat_real.head()

,name,smiles,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_mean,duration_mean,...,avg_mass,logP,logS,PSA,HBD,HBA,rot_bonds,rings,pKa_acid,pKa_base
0,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,moderate,mid,medium,medium,40.0,6.0,...,30.0061,-0.35,NaN,34.14,0.0,1.0,0.0,0.0,NaN,-2.90
1,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,strong,high,medium,medium,60.0,2.0,...,146.1445,-4.00,-0.17,106.41,3.0,4.0,4.0,0.0,2.15,9.31
2,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,4.0,...,392.5180,-0.50,-6.80,46.53,1.0,2.0,5.0,5.0,11.05,-4.50
3,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,moderate,mid,medium,low,40.0,4.5,...,267.2413,-2.10,-1.30,139.54,4.0,8.0,2.0,3.0,12.45,3.94
4,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,strong,high,short,medium,60.0,1.0,...,392.4611,1.68,-3.90,94.83,3.0,5.0,2.0,4.0,12.42,-3.30


In [24]:
data_final=pd.read_csv('final_model_data_with_mass_balance.csv')

In [25]:
data_final.head()

,name,smiles,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_mean,duration_mean,...,HBA,rot_bonds,rings,pKa_acid,pKa_base,smb_mass_balance,amb_mass_balance,amb_deficiency,rmb_mass_balance,rmb_deficiency
0,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,moderate,mid,medium,medium,40.0,6.0,...,1.0,0.0,0.0,NaN,-2.90,95.91,95.91,4.09,41.654779,58.345221
1,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,strong,high,medium,medium,60.0,2.0,...,4.0,4.0,0.0,2.15,9.31,93.07,93.07,6.93,42.346090,57.653910
2,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,4.0,...,2.0,5.0,5.0,11.05,-4.50,96.82,96.82,3.18,41.001855,58.998145
3,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,moderate,mid,medium,low,40.0,4.5,...,8.0,2.0,3.0,12.45,3.94,96.66,96.66,3.34,41.506130,58.493870
4,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,strong,high,short,medium,60.0,1.0,...,5.0,2.0,4.0,12.42,-3.30,92.98,92.98,7.02,42.459016,57.540984


In [26]:
mass_balance_data=pd.read_csv('final_model_data_with_mass_balance.csv')

In [27]:
mass_balance_data.head()

,name,smiles,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_mean,duration_mean,...,HBA,rot_bonds,rings,pKa_acid,pKa_base,smb_mass_balance,amb_mass_balance,amb_deficiency,rmb_mass_balance,rmb_deficiency
0,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,moderate,mid,medium,medium,40.0,6.0,...,1.0,0.0,0.0,NaN,-2.90,95.91,95.91,4.09,41.654779,58.345221
1,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,strong,high,medium,medium,60.0,2.0,...,4.0,4.0,0.0,2.15,9.31,93.07,93.07,6.93,42.346090,57.653910
2,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,4.0,...,2.0,5.0,5.0,11.05,-4.50,96.82,96.82,3.18,41.001855,58.998145
3,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,moderate,mid,medium,low,40.0,4.5,...,8.0,2.0,3.0,12.45,3.94,96.66,96.66,3.34,41.506130,58.493870
4,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,strong,high,short,medium,60.0,1.0,...,5.0,2.0,4.0,12.42,-3.30,92.98,92.98,7.02,42.459016,57.540984


In [2]:
import numpy as np
import pandas as pd


In [3]:
data=pd.read_csv('model_predictions_complete.csv')

In [4]:
data.head()

,drugbank_id,name,smiles,stress_type,reagent,reagent.1,severity,severity.1,temp_bin,temp_bin.1,...,predicted_degradant_percent,predicted_degradant_percent.1,freq_weight_mean.2,freq_weight_mean.3,predicted_api_loss_percent.2,predicted_api_loss_percent.3,predicted_degradant_percent.2,predicted_degradant_percent.3,api_prediction_error,degradant_prediction_error
0,DB00435,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,HCl,moderate,moderate,mid,mid,...,5.5,5.5,0.015652,0.015652,11.0,11.0,5.5,5.5,3.73,2.43
1,DB00130,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,NaOH,strong,strong,high,high,...,5.0,5.0,0.013043,0.013043,12.0,12.0,5.0,5.0,0.46,0.30
2,DB00209,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,HCl,moderate,moderate,high,high,...,3.5,3.5,0.016957,0.016957,7.0,7.0,3.5,3.5,1.76,1.39
3,DB00194,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,NaOH,moderate,moderate,mid,mid,...,2.0,2.0,0.011739,0.011739,5.0,5.0,2.0,2.0,0.67,0.38
4,DB00443,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,NaOH,strong,strong,high,high,...,6.0,6.0,0.020870,0.020870,15.0,15.0,6.0,6.0,2.64,0.74


In [6]:
data.columns

Index(['drugbank_id', 'name', 'smiles', 'stress_type', 'reagent', 'reagent.1',
       'severity', 'severity.1', 'temp_bin', 'temp_bin.1', 'duration_bin',
       'duration_bin.1', 'strength_bin', 'strength_bin.1', 'temperature_mean',
       'duration_mean', 'strength_mean', 'pH_mean', 'initial_api_percent',
       'initial_degradant_percent', 'true_api_loss_percent',
       'true_degradant_percent', 'model_predicted_api_loss_percent',
       'model_predicted_degradant_percent', 'Unnamed: 0.3', 'Unnamed: 0.1',
       'Unnamed: 0', 'avg_mass', 'logP', 'logS', 'PSA', 'HBD', 'HBA',
       'rot_bonds', 'rings', 'pKa_acid', 'pKa_base', 'phys_charge',
       'temperature_min', 'temperature_max', 'duration_min', 'duration_max',
       'strength_min', 'strength_max', 'pH_min', 'pH_max', 'freq_weight_mean',
       'freq_weight_mean.1', 'light_lux_hr_applied', 'uv_Wh_m2_applied',
       'humidity_RH_applied', 'Unnamed: 0.2', 'predicted_api_loss_percent',
       'predicted_api_loss_percent.1', 'pre

In [7]:
data=data.drop(columns=['reagent.1','severity.1','temp_bin.1','duration_bin.1','strength_bin.1','predicted_api_loss_percent.1'])

In [8]:
data=data.drop(columns=['predicted_api_loss_percent','predicted_degradant_percent', 'api_prediction_error','degradant_prediction_error'])

In [9]:
data.columns

Index(['drugbank_id', 'name', 'smiles', 'stress_type', 'reagent', 'severity',
       'temp_bin', 'duration_bin', 'strength_bin', 'temperature_mean',
       'duration_mean', 'strength_mean', 'pH_mean', 'initial_api_percent',
       'initial_degradant_percent', 'true_api_loss_percent',
       'true_degradant_percent', 'model_predicted_api_loss_percent',
       'model_predicted_degradant_percent', 'Unnamed: 0.3', 'Unnamed: 0.1',
       'Unnamed: 0', 'avg_mass', 'logP', 'logS', 'PSA', 'HBD', 'HBA',
       'rot_bonds', 'rings', 'pKa_acid', 'pKa_base', 'phys_charge',
       'temperature_min', 'temperature_max', 'duration_min', 'duration_max',
       'strength_min', 'strength_max', 'pH_min', 'pH_max', 'freq_weight_mean',
       'freq_weight_mean.1', 'light_lux_hr_applied', 'uv_Wh_m2_applied',
       'humidity_RH_applied', 'Unnamed: 0.2', 'predicted_degradant_percent.1',
       'freq_weight_mean.2', 'freq_weight_mean.3',
       'predicted_api_loss_percent.2', 'predicted_api_loss_percent.3',
  

In [10]:
data=data.drop(columns=['Unnamed: 0.2', 'predicted_degradant_percent.1',
       'freq_weight_mean.2', 'freq_weight_mean.3',
       'predicted_api_loss_percent.2', 'predicted_api_loss_percent.3',
       'predicted_degradant_percent.2', 'predicted_degradant_percent.3','Unnamed: 0.3', 'Unnamed: 0.1',
       'Unnamed: 0'])

In [11]:
data.head()

,drugbank_id,name,smiles,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_mean,...,duration_max,strength_min,strength_max,pH_min,pH_max,freq_weight_mean,freq_weight_mean.1,light_lux_hr_applied,uv_Wh_m2_applied,humidity_RH_applied
0,DB00435,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,moderate,mid,medium,medium,40.0,...,6.0,0.10,0.10,1.2,1.2,0.015652,0.015652,0,0,0
1,DB00130,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,strong,high,medium,medium,60.0,...,2.0,0.10,0.10,13.0,13.0,0.013043,0.013043,0,0,0
2,DB00209,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,...,6.0,0.10,0.10,1.2,1.2,0.016957,0.016957,0,0,0
3,DB00194,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,moderate,mid,medium,low,40.0,...,6.0,0.01,0.05,12.0,12.5,0.011739,0.011739,0,0,0
4,DB00443,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,strong,high,short,medium,60.0,...,1.0,0.10,0.10,13.0,13.0,0.020870,0.020870,0,0,0


In [13]:
data.columns

Index(['drugbank_id', 'name', 'smiles', 'stress_type', 'reagent', 'severity',
       'temp_bin', 'duration_bin', 'strength_bin', 'temperature_mean',
       'duration_mean', 'strength_mean', 'pH_mean', 'initial_api_percent',
       'initial_degradant_percent', 'true_api_loss_percent',
       'true_degradant_percent', 'model_predicted_api_loss_percent',
       'model_predicted_degradant_percent', 'avg_mass', 'logP', 'logS', 'PSA',
       'HBD', 'HBA', 'rot_bonds', 'rings', 'pKa_acid', 'pKa_base',
       'phys_charge', 'temperature_min', 'temperature_max', 'duration_min',
       'duration_max', 'strength_min', 'strength_max', 'pH_min', 'pH_max',
       'freq_weight_mean', 'freq_weight_mean.1', 'light_lux_hr_applied',
       'uv_Wh_m2_applied', 'humidity_RH_applied'],
      dtype='object')

In [14]:
data=data.drop(columns=['temperature_min', 'temperature_max', 'duration_min',
       'duration_max', 'strength_min', 'strength_max', 'pH_min', 'pH_max',
       'freq_weight_mean', 'freq_weight_mean.1', 'light_lux_hr_applied',
       'uv_Wh_m2_applied', 'humidity_RH_applied'])

In [15]:
data.head()

,drugbank_id,name,smiles,stress_type,reagent,severity,temp_bin,duration_bin,strength_bin,temperature_mean,...,logP,logS,PSA,HBD,HBA,rot_bonds,rings,pKa_acid,pKa_base,phys_charge
0,DB00435,Nitric Oxide,[N]=O,acid_hydrolysis,HCl,moderate,mid,medium,medium,40.0,...,-0.35,NaN,34.14,0.0,1.0,0.0,0.0,NaN,-2.90,0.0
1,DB00130,L-Glutamine,N[C@@H](CCC(N)=O)C(O)=O,base_hydrolysis,NaOH,strong,high,medium,medium,60.0,...,-4.00,-0.17,106.41,3.0,4.0,4.0,0.0,2.15,9.31,0.0
2,DB00209,Trospium,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)(C1=...,acid_hydrolysis,HCl,moderate,high,medium,medium,60.0,...,-0.50,-6.80,46.53,1.0,2.0,5.0,5.0,11.05,-4.50,1.0
3,DB00194,Vidarabine,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](CO)[C@@H](O)[C...,base_hydrolysis,NaOH,moderate,mid,medium,low,40.0,...,-2.10,-1.30,139.54,4.0,8.0,2.0,3.0,12.45,3.94,0.0
4,DB00443,Betamethasone,[H][C@@]12C[C@H](C)[C@](O)(C(=O)CO)[C@@]1(C)C[...,base_hydrolysis,NaOH,strong,high,short,medium,60.0,...,1.68,-3.90,94.83,3.0,5.0,2.0,4.0,12.42,-3.30,0.0


In [16]:
data=data.drop(columns=['drugbank_id','phys_charge'])

In [17]:
data.shape

(1000, 28)

In [18]:
data.to_csv('cleaned_model_predictions.csv',index=False)